# Exercise 1

In [68]:
# Importing the libraries

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import split

In [69]:
import findspark

findspark.init()

In [70]:
# Creating a new session on Spark

spark = SparkSession.builder.appName("ClarityAI_test").getOrCreate()

In [71]:
# Read the input file provided

df = spark.read.text(r"C:\Users\34676\Desktop\prueba_tecnica_clarityAI\input-file.txt")

In [72]:
df

DataFrame[value: string]

The dataframe only contains one column with all the data, but we need to have the timestamp,the first host and the second host in separate columns so that we can later select the columns with an sql query.

In [73]:
# Creating three columns with the data separated so that we can work with it later

df1 = df.withColumn('timestamp', split(df['value'], ' ').getItem(0)) \
       .withColumn('from', split(df['value'], ' ').getItem(1))\
       .withColumn('to', split(df['value'], ' ').getItem(2))

In [74]:
df1.show(truncate=False)

+--------------------------------+-------------+-----------+--------+
|value                           |timestamp    |from       |to      |
+--------------------------------+-------------+-----------+--------+
|1565647204351 Aadvik Matina     |1565647204351|Aadvik     |Matina  |
|1565647205599 Keimy Dmetri      |1565647205599|Keimy      |Dmetri  |
|1565647212986 Tyreonna Rehgan   |1565647212986|Tyreonna   |Rehgan  |
|1565647228897 Heera Eron        |1565647228897|Heera      |Eron    |
|1565647246869 Jeremyah Morrigan |1565647246869|Jeremyah   |Morrigan|
|1565647247170 Khiem Tailee      |1565647247170|Khiem      |Tailee  |
|1565647256008 Remiel Jadon      |1565647256008|Remiel     |Jadon   |
|1565647260788 Monet Jarreth     |1565647260788|Monet      |Jarreth |
|1565647264445 Jil Cerena        |1565647264445|Jil        |Cerena  |
|1565647268712 Naetochukwu Kallan|1565647268712|Naetochukwu|Kallan  |
|1565647289553 Jahmani Markena   |1565647289553|Jahmani    |Markena |
|1565647300338 Davie

Then we create a temporary table that stores the data we are going to retrieve with the sql query.

In [75]:
#Create a temporary table

df1.registerTempTable("Logs")

In [76]:
# Proving the temporary table has been created correctly

spark.sql("SELECT * FROM Logs").show()

+--------------------+-------------+-----------+--------+
|               value|    timestamp|       from|      to|
+--------------------+-------------+-----------+--------+
|1565647204351 Aad...|1565647204351|     Aadvik|  Matina|
|1565647205599 Kei...|1565647205599|      Keimy|  Dmetri|
|1565647212986 Tyr...|1565647212986|   Tyreonna|  Rehgan|
|1565647228897 Hee...|1565647228897|      Heera|    Eron|
|1565647246869 Jer...|1565647246869|   Jeremyah|Morrigan|
|1565647247170 Khi...|1565647247170|      Khiem|  Tailee|
|1565647256008 Rem...|1565647256008|     Remiel|   Jadon|
|1565647260788 Mon...|1565647260788|      Monet| Jarreth|
|1565647264445 Jil...|1565647264445|        Jil|  Cerena|
|1565647268712 Nae...|1565647268712|Naetochukwu|  Kallan|
|1565647289553 Jah...|1565647289553|    Jahmani| Markena|
|1565647300338 Dav...|1565647300338|      Davie| Nafeesa|
|1565647303287 Ant...|1565647303287|    Antwine|Grantham|
|1565647309932 Nyi...|1565647309932|    Nyiasia|Zoeylynn|
|1565647313867

First I tried passing the data to the query manually, so that I could test if it worked correctly.

In [77]:
configs = {"init_datetime":1565647204351,
           "end_datetime":1565647204352,
           "to_host":'Alyaa'}

Q1 = spark.sql("""SELECT timestamp, from FROM Logs 
                  WHERE timestamp>{init_datetime} AND timestamp>{end_datetime} AND to='{to_host}'
               """.format(**configs)).show()

+-------------+--------+
|    timestamp|    from|
+-------------+--------+
|1565647368307|  Mehkai|
|1565650799255|  Donnis|
|1565663573006|  Careli|
|1565685686037| Irielle|
|1565697613185| Mugilan|
|1565711104391|  Nyarai|
|1565715430365|Adreyona|
+-------------+--------+



Once I had tested that the query worked well I passed the data to the query with the user input.

In [78]:
init_datetime=input("Write an initial time: ")

Write an initial time: 1565647368307


In [79]:
end_datetime=input("Write a final time: ")

Write a final time: 1565650799255


In [80]:
to_host=input("Write a host: ")

Write a host: Alyaa


In [81]:
configs = {"init_datetime":init_datetime,
           "end_datetime":end_datetime,
           "to_host":to_host}
Q1 = spark.sql("""SELECT timestamp, from FROM Logs 
                  WHERE timestamp>{init_datetime} AND timestamp>{end_datetime} AND to='{to_host}'
               """.format(**configs)).show()

+-------------+--------+
|    timestamp|    from|
+-------------+--------+
|1565663573006|  Careli|
|1565685686037| Irielle|
|1565697613185| Mugilan|
|1565711104391|  Nyarai|
|1565715430365|Adreyona|
+-------------+--------+



The query returns a list of hostnames connected to the host, in this example Alyaa, in between the inicial and the end given time.

# Exercise 2

In [33]:
# Import the libraries

from datetime import datetime

In [62]:
from pyspark.sql.functions import to_date, col, hour, minute, second

In [37]:
# I tried to define a function that gets the hours from timestamp format and creates a file each hour.

def hourly():
    df = spark.read.text(r"C:\Users\34676\Desktop\prueba_tecnica_clarityAI\input-file.txt", "r")
    df1 = df.withColumn('timestamp', split(df['value'], ' ').getItem(0)) \
       .withColumn('from', split(df['value'], ' ').getItem(1))\
       .withColumn('to', split(df['value'], ' ').getItem(2))
    data = df1.select("timestamp")
    d = datetime.strptime(data, '%Y-%m-%d %H:%M:%S')
    df3['hour'] = d.hour
    if data==0:
        print(data)
        
        
hourly()
           

In [63]:
# I was trying to get the hours from the timestamp column

df1.withColumn("timestamp",hour(col("timestamp"))).show()

+--------------------+---------+-----------+--------+
|               value|timestamp|       from|      to|
+--------------------+---------+-----------+--------+
|1565647204351 Aad...|     null|     Aadvik|  Matina|
|1565647205599 Kei...|     null|      Keimy|  Dmetri|
|1565647212986 Tyr...|     null|   Tyreonna|  Rehgan|
|1565647228897 Hee...|     null|      Heera|    Eron|
|1565647246869 Jer...|     null|   Jeremyah|Morrigan|
|1565647247170 Khi...|     null|      Khiem|  Tailee|
|1565647256008 Rem...|     null|     Remiel|   Jadon|
|1565647260788 Mon...|     null|      Monet| Jarreth|
|1565647264445 Jil...|     null|        Jil|  Cerena|
|1565647268712 Nae...|     null|Naetochukwu|  Kallan|
|1565647289553 Jah...|     null|    Jahmani| Markena|
|1565647300338 Dav...|     null|      Davie| Nafeesa|
|1565647303287 Ant...|     null|    Antwine|Grantham|
|1565647309932 Nyi...|     null|    Nyiasia|Zoeylynn|
|1565647313867 Kis...|     null|   Kishauna|    Liem|
|1565647318266 Can...|     n